In [12]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['USA'], limit_per_country=5000):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Strong Buy (1)',
                'Country': country,
                'Average Volume': 'Over 2M', # Added volume filter for liquidity
                'Market Cap.': '+Small (over $300mln)'
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 1.5 (Strong Buys)
        print(f"Filtering out 'Hold' or worse (Rating > 1.5)...")
        combined_df = combined_df[combined_df[recom_col] <= 1.0]
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    # Ensure Price is numeric (Crucial for sorting in Data Viewer)
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(top_picks)} candidates.")
    print("Variable 'top_picks' is ready for the Data Viewer.")

else:
    print("DataFrame is empty.")

Fetching candidates for: USA...
[Info] loading page [###########################---] 10/11 
Success! Found 207 candidates.
Variable 'top_picks' is ready for the Data Viewer.


In [13]:
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    mid_picks = top_picks[(top_picks['Price'] > 5) & (top_picks['Price'] < 30)]

    print(f"\nFound {len(mid_picks)} stocks between $5 and $30.")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 110 stocks between $5 and $30.


In [14]:
# filter for price range less than $5 a bit gambly
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    cheap_picks = top_picks[(top_picks['Price'] < 5)]

    print(f"\nFound {len(cheap_picks)} stocks less than $5")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 28 stocks less than $5


In [15]:
# filter for price range less than $5 a bit gambly
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    Main_TFSA_picks = top_picks[(top_picks['Price'] > 20)]

    print(f"\nFound {len(Main_TFSA_picks)} stocks greater than $20.")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 98 stocks greater than $20.


In [16]:
"""
import yfinance as yf
import time

# Loop through EVERY symbol in the list (removed the [:3])
for symbol in ticker_list:
    
    # 1. Fetch data
    stock = yf.Ticker(symbol)
    print(f"\n--- Checking {symbol} ---")
    
    try:
        # 2. Get the upgrades/downgrades
        upgrades = stock.upgrades_downgrades
        
        if not upgrades.empty:
            # Filter for 2025 actions using the .index fix
            recent = upgrades[upgrades.index >= '2025-11-01']
            
            if not recent.empty:
                print(recent)
            else:
                print(f"No actions for {symbol} in 2025.")
        else:
            print(f"No analyst history found for {symbol}.")
            
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

    # 3. CRITICAL: Sleep for 1 second to avoid getting banned
    time.sleep(1)
"""

'\nimport yfinance as yf\nimport time\n\n# Loop through EVERY symbol in the list (removed the [:3])\nfor symbol in ticker_list:\n    \n    # 1. Fetch data\n    stock = yf.Ticker(symbol)\n    print(f"\n--- Checking {symbol} ---")\n    \n    try:\n        # 2. Get the upgrades/downgrades\n        upgrades = stock.upgrades_downgrades\n        \n        if not upgrades.empty:\n            # Filter for 2025 actions using the .index fix\n            recent = upgrades[upgrades.index >= \'2025-11-01\']\n            \n            if not recent.empty:\n                print(recent)\n            else:\n                print(f"No actions for {symbol} in 2025.")\n        else:\n            print(f"No analyst history found for {symbol}.")\n            \n    except Exception as e:\n        print(f"Error fetching {symbol}: {e}")\n\n    # 3. CRITICAL: Sleep for 1 second to avoid getting banned\n    time.sleep(1)\n'

In [17]:
import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                # Check if data has MultiIndex columns (nested structure)
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    # Single level columns (Flat)
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                # Print the actual error so we know what's wrong
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    # We check if df_yf is empty, but we allow df_finviz to have data (Outer Merge)
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            # If yfinance failed completely, just return Finviz data
            print("   Warning: Returning Finviz data only (yfinance failed).")
            master_df = df_finviz
            
        cols = ['Ticker', 'Price', 'Change_%', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        # Filter only columns that exist
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    # Ensure numeric for sorting
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 1 stocks ---
1. Fetching Analyst Ratings from Finviz...


2. Fetching Price & Volatility from yfinance...


C:\Users\jdcc3\AppData\Local\Temp\ipykernel_7692\3538496220.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
0,GRND,12.85,0.16,-48.87,1.40,21.75,0.55,3.23
